In [ ]:
'''
Instructions to run the benchmarks:

Use Python 3 to run this script
For SymEngine benchmarks compile with -DWITH_TCMALLOC=yes -DWITH_FLINT=yes -DINTEGER_CLASS=flint
For GiNaC benchmarks compile with the same CXX flags as SymEngine
Move (or symlink) all the benchmarks from symengine/symengine repo and symengine/symengine.py repo to this folder
'''

from subprocess import Popen, PIPE, TimeoutExpired
import json
import os
import signal

test1 = ["expand2", list(range(5, 20, 2))]
test2 = ["expand6", list(range(100, 1100, 100))]
test4 = ["expand6b", list(range(100, 1100, 100))]
test3 = ["expand7", list(range(10, 50, 3))]

tests = [test4, test3, test2 test1]
for test in tests:
    name = test[0]
    param = test[1][:]
    for b in range(5):
        lib = ["./%s %s","./%s_ginac %s", "./%s.m %s", "./%s.py %s", "maple -q %s.mpl -D n=%s"][b]
        time = [float('inf')] * len(test[1])
        for i in range(len(param)):
            test_name = lib % (name, param[i])
            command = 'LD_PRELOAD=/usr/lib/libtcmalloc.so '+test_name
            with Popen(command, shell=True, stdout=PIPE, preexec_fn=os.setsid) as process:
                try:
                    output = process.communicate(timeout=120)[0]
                    time[i] = float(output.splitlines()[0].decode('utf-8').replace("ms", "").strip())
                    time[i] = max(0.01, time[i])
                except TimeoutExpired:
                    os.killpg(process.pid, signal.SIGINT) # send signal to the process group
                    break
            print(test_name, time[i])
        test.append(time)
        print(time)
with open('data.json', 'w') as f:
    f.write(json.dumps(tests))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import json
import numpy as np
with open('data.json', 'r') as f:
    tests = json.loads(f.read())
for test in tests:
    name = test[0]
    param = np.log(test[1][:])
    plt.plot(param, np.log(test[2]), 'g', label="SymEngine")
    plt.plot(param, np.log(test[3]), 'b', label="GiNaC")
    plt.plot(param, np.log(test[4]), 'r', label="Mathematica")
    plt.plot(param, np.log(test[5]), 'y', label="SymPy")
    plt.plot(param, np.log(test[6]), 'c', label="Maple")
    plt.xlabel('Parameter')
    plt.ylabel('Time in ms')
    plt.title("Benchmark "+name)
    lgd = plt.legend(loc='center right', bbox_to_anchor=(1.4, 0.5))
    plt.show()